# SEP Topic-Taxonomy Integration

This notebook loads taxonomy categories from the InPhO database and builds up a JSON structure of the tree heirarchy of the items and the links between them. The tree is based on [https://inpho.cogs.indiana.edu/taxonomy]

In order to do this with another type of data (for example, dynasty mapping), you will need to generate a similar link-topic strucutre for the documents. Rather than using the depth as levels in the tree, I would use just the start year for the dynasty. If this is working, then the topics on the left should sort so there's little overlapping and you might be able to read the topic ordering from top to bototom as earliest to latest.

In [11]:
%pylab inline
from sepcurrent import *

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [12]:
from inpho.model import *

In [13]:
sep_ideas = Session.query(Idea).filter(Idea.sep_dir!='')

In [14]:
from collections import defaultdict
parent = dict()
node_entries = defaultdict(list)

def get_descendants(node):
    node_entries[node]

In [15]:
idea = Session.query(Idea).get(646)

In [16]:
idea.nodes

[<Node 20239: epistemology>]

In [17]:
idea.sep_dir

'epistemology'

In [18]:
node = idea.nodes[0]
node

<Node 20239: epistemology>

In [20]:
root_nodes = Session.query(Node).filter(Node.parent_id==None).all()

node_entries = defaultdict(list)
node_entries[0] = [c.idea.sep_dir for c in root_nodes if c.idea.sep_dir]
print node_entries[0]

def populate_descendants(nodes):
    for node in nodes:
        concepts = [node]
        concepts.extend(node.idea.instances)
        concepts.extend(child.idea for child in node.children)
        concepts = [c.sep_dir for c in concepts if c.sep_dir]
        # add entries to list
        if concepts:
            node_entries[node].extend(concepts)
        
        # recur
        if node.children:
            populate_descendants(node.children)

populate_descendants(root_nodes)

['biology-philosophy', 'philosophy-mathematics', 'computer-science', 'epistemology', 'psychiatry', 'chemistry', 'philosophy-religion', 'metaphysics', 'economics']


In [21]:
def get_descendants(node):
    descendants = node_entries[node]
    for child in node.children:
        descendants.extend(get_descendants(child))
    return set(descendants)

In [22]:
for node in root_nodes:
    print node.idea.label, len(get_descendants(node))

aesthetics and philosophy of art 15
history of philosophy 20
social and political philosophy 32
probability and statistics 4
philosophy of physics 29
logic 57
philosophy of mind 64
philosophy of language 37
feminist philosophy 10
philosophy of biology 21
philosophy of action 0
philosophy of mathematics 7
philosophy of law 13
philosophy of computer science 6
ethics 43
epistemology 24
psychology and psychiatry 0
philosophy of sociology 1
philosophy of chemistry 0
philosophy of medicine 0
philosophy of religion 36
philosophy of science 15
metaphysics 48
philosophy of economics 2


In [100]:
v = lda_v[120]
overall_topics = v.aggregate_doc_topics(v.labels)

def get_topics(node):
    entries = [entry for entry in get_descendants(node) if entry in v.labels]
    if entries:
        area_topics = v.aggregate_doc_topics(entries)
        significant_topics = area_topics['value'] > (overall_topics[area_topics['i']]['value'] ** 0.5)
        area_topics = area_topics[significant_topics]
        if np.array(significant_topics).any():
            tops = list(np.array(area_topics)['i'])
            return tops

    return []

def get_raw_topic_proportion(tops, node):
    entries = [entry for entry in get_descendants(node) if entry in v.labels]
    if entries:
        entry_idxs = np.in1d(v.labels, entries)
        topics = v.model.top_doc[tops][:,entry_idxs].sum(axis=1)
        insignificant_topics = topics < (overall_topics[tops]['value'] ** 0.5)
        topics[insignificant_topics] = 0.
        return topics
        

from itertools import chain
node_topics = defaultdict(list)
for node in root_nodes:
    node_topics[node] = get_topics(node)
print node_topics

set(chain(*node_topics.values()))

defaultdict(<type 'list'>, {<Node 20177: feminist philosophy>: [111, 78, 112], <Node 20185: philosophy of biology>: [47, 50, 64], <Node 20356: metaphysics>: [87], <Node 20191: philosophy of action>: [], <Node 20193: philosophy of mathematics>: [29, 54, 91], <Node 20112: aesthetics and philosophy of art>: [66, 87, 103], <Node 20203: philosophy of law>: [93, 12, 117, 87], <Node 20228: philosophy of computer science>: [50, 63], <Node 20137: probability and statistics>: [99, 72, 91, 110], <Node 20238: ethics>: [87, 102], <Node 20239: epistemology>: [107, 87], <Node 20268: psychology and psychiatry>: [], <Node 20367: philosophy of economics>: [90, 43, 93, 50, 78], <Node 20273: philosophy of sociology>: [96, 78, 117], <Node 20113: history of philosophy>: [], <Node 20283: philosophy of chemistry>: [], <Node 20284: philosophy of medicine>: [], <Node 20291: philosophy of religion>: [69, 87], <Node 20146: philosophy of physics>: [81, 56, 91, 87], <Node 20293: philosophy of science>: [95, 87], <N

{12,
 29,
 40,
 42,
 43,
 47,
 50,
 54,
 56,
 63,
 64,
 65,
 66,
 69,
 72,
 78,
 81,
 86,
 87,
 90,
 91,
 93,
 95,
 96,
 99,
 102,
 103,
 107,
 110,
 111,
 112,
 117}

## The big functions
This is what generates the "nodes" and "links" JSON structure expected by the D3 Sankey visualization. Note that at the top is a list of every item, and after that is every link between taxonomy levels. The basic schema is:

```
{
    // The nodes list
    'nodes' : [
        {
            'name' : "node name", 
            'color' : "#ff0000', 
            'depth' : 2
        }, ... ]
    // the links list
    'links' : [
        {
            "color": "#f781bf", 
             "source": "philosophy of art", 
             "target": "artistic form", 
             "topic": "T103", 
             "value": 421.00999999999999
         }, ... ]
}
```

Notice that `"topic"` refers to the topic number with 'T' before it - you'll need to set this up to make the code work since JavaScript will recast `'103'` to an integer `103` which causes all kinds of issues with indexing.

In [110]:
import topicexplorer.lib.color as color
topic_color = dict(get_topic_colors(v))

# create links
links = []
def add_child_links(node, root):
    # add 1 level of children
    for child in node.children:
        if get_descendants(child):
            for topic, prop in zip(node_topics[root], get_raw_topic_proportion(node_topics[root], child)):
                if prop:
                    links.append({'source': node.label,
                                  'target': child.label,
                                  'value' : prop,
                                  'color' : color.rgb2hex(topic_color[topic]),
                                  'topic' : 'T{}'.format(topic)
                             })
        add_child_links(child, root)

for node in root_nodes:
    if node_topics[node]:
        for topic, prop in zip(node_topics[node], get_raw_topic_proportion(node_topics[node], node)):
            if prop:
                links.append({'source': 'T{}'.format(topic),
                              'target': node.label,
                              'value' : prop,
                              'color' : color.rgb2hex(topic_color[topic]),
                              'topic' : 'T{}'.format(topic)
                             })
        add_child_links(node, node)

        
# create nodes
nodes = []
added_nodes = root_nodes[:]
topics = v.topics()

def add_child_nodes(node, depth=2, root=None, first_page=True):
    for child in node.children:
            if get_descendants(child) and child not in added_nodes and np.any(get_raw_topic_proportion(node_topics[root], child) > 0):
                nodes.append({'name' : child.label, 'depth' : depth, 'inpho_id' : child.ID})
                add_child_nodes(child, depth + 1, root)

for topic in set(chain(*node_topics.values())):
    nodes.append({'name' : 'T{}'.format(topic),
                 'color' : color.rgb2hex(topic_color[topic]),
                 'words' : ', '.join(topics[topic][:10]['word']),
                 'depth' : 0})



for node in root_nodes:
    if node_topics[node]:
        nodes.append({'name' : node.label, 'depth' : 1, 'inpho_id' : node.ID})
        add_child_nodes(node, 2, node)
        
print len(links), len(nodes)

import json
print json.dumps({'links' : links, 'nodes' : nodes})

267 171
{"nodes": [{"color": "#ff8000", "depth": 0, "name": "T12", "words": "punishment, harm, criminal, forgiveness, wrong, torture, civil, justice, crime, morally"}, {"color": "#fffe33", "depth": 0, "name": "T29", "words": "sets, numbers, axioms, axiom, cardinal, zfc, large, dedekind, cantor, paradox"}, {"color": "#e41a1c", "depth": 0, "name": "T40", "words": "rights, property, children, child, individuals, nozick, free, private, duties, legal"}, {"color": "#4daf4a", "depth": 0, "name": "T42", "words": "propositions, proposition, sentences, sentence, context, false, worlds, propositional, semantic, content"}, {"color": "#974fa2", "depth": 0, "name": "T43", "words": "justice, equality, equal, society, opportunity, health, economic, distributive, distribution, resources"}, {"color": "#f781bf", "depth": 0, "name": "T47", "words": "selection, evolutionary, species, biology, biological, genetic, genes, evolution, organisms, gene"}, {"color": "#4daf4a", "depth": 0, "name": "T50", "words": 

In [ ]:
def wikiscraper(user):
    """ This function manages user state. """
    user_url = 'http://...'
    links = getedits(url)
    return links
    
    
def getedits(url, follow=1):
    """ This function manages page state. """
    """ url -> list of links """
    # init your skip state
    skipped_older_500 = False
    links = []
   
    # do your beautiful soup
    
    # this if works on each link
    for link in page:
        if text == 'older 500' and not skipped_older_500:
            skipped_older_500 = True
        elif text == 'older 500' and skipped_older_500 and follow:
            links.extend(getedits(new_url, follow=follow-1))
        else:
            # add the link to the list
            
    return links
    
        
    

In [102]:
area_topics = np.array(v.aggregate_doc_topics(node_entries[root_nodes[0]]))
base = np.array(overall_topics[area_topics['i']]['value'])
np.log(area_topics['value']) / np.log(base)

array([ 0.310272  ,  0.48184085,  0.45513919,  0.47858505,  0.60753701,
        0.61372684,  0.71403141,  0.69442363,  0.71759592,  0.65502096,
        0.88270853,  0.75983752,  1.0460637 ,  0.81904679,  0.79499943,
        0.70368536,  0.79850875,  0.78913044,  0.69340608,  0.78039998,
        0.76607882,  0.7796588 ,  0.81655627,  0.93597587,  0.99280379,
        1.37648754,  1.41123615,  1.09827117,  0.9942905 ,  0.97138368,
        1.00145027,  1.17372277,  1.07846883,  0.99536113,  0.94743214,
        0.9638703 ,  1.2770373 ,  1.15346257,  1.3388641 ,  1.28751205,
        1.08779091,  1.1909664 ,  1.38669517,  1.09890238,  1.27834918,
        1.06514859,  0.97803753,  1.04669854,  1.19690388,  1.18520655,
        1.23681951,  1.16943073,  1.24969881,  1.1170645 ,  1.33939471,
        1.20815952,  1.33149258,  1.29014342,  1.69636798,  1.80528124,
        1.29883466,  2.44691486,  1.27657855,  1.27744263,  1.20266154,
        1.51462656,  1.29605184,  1.27266093,  1.36980883,  1.45